1\. Write a function that converts number representation (bin<->dec<->hex)

In [94]:
def bin_to_dec(x):
    dec = 0
    for i,j in enumerate(list(str(x))[::-1]):
        dec += int(j)*2**(int(i))
    return dec

def dec_to_bin(x):
    binary = ""
    while x:
        if x & 1 == 1:
            binary += "1"
        else:
            binary += "0"
        x //= 2
    return binary[::-1]

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [44]:
def to_float(x, debug=False):
    """
    Conversion from any given integer, treated as a 32 bit word, to the correponding single-precision floating-point number.
    No special cases (Nan, inf, ...) are considered.
    """
    read_sign = 1<<31 # '0b10000000000000000000000000000000'
    sign = (-1)**((read_sign & x)>>31)
    
    read_exp = 255<<23 # '0b1111111100000000000000000000000'
    exp = (x & read_exp)>>23
    exp = 2**(exp-127)
    
    read_mantissa = 8388607 # x23 1's '0b11111111111111111111111'
    mantissa = x & read_mantissa
    # fraction computation
    fraction = 0.
    for j,i in enumerate(list(bin(mantissa)[2:]), start=1):
        fraction += int(i)*2**(-j)
    mantissa = 1 + fraction
    
    if debug == True:
        print('Sign: ',sign)
        print('Exp: ',exp)
        print('Mantissa: ',mantissa,' from ', bin(x & read_mantissa))
    
    return sign*exp*mantissa
        

In [45]:
#pi = "0 10000000 10010010000111111011011"
int_pi = int("01000000010010010000111111011011",2)
to_float(int_pi,debug=True)


Sign:  1
Exp:  2
Mantissa:  1.5707963705062866  from  0b10010010000111111011011


3.1415927410125732

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [11]:
under = 1.
counter = 0

while under != 0:
    under/=2.
    counter+=1
print('Underflow, numbers of iterations: ', counter, '; result ',under)
print(format(under,'.100f'))

over = 1.
counter = 0
while over+1 != over:
    over*=2.
    counter+=1
print('Overflow, numbers of iterations: ', counter, '; result ', over)    

Underflow, numbers of iterations:  1075 ; result  0.0
0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Overflow, numbers of iterations:  53 ; result  9007199254740992.0


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [5]:
a = 1.
b = 1.
counter = 0

while a+b != a:
    b/=10.
    counter+=1
print('Machine Precision: ', b, '= (1/10)**', counter)

Machine Precision:  1.0000000000000001e-16 = (1/10)** 16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [23]:
import math
from decimal import Decimal as D

def quad_eq(a,b,c):
    delta = math.sqrt(b**2-4*a*c)
    return ((-b+delta)/(2*a),(-b-delta)/(2*a))

def quad_eq_v2(a,b,c):
    delta = math.sqrt(b**2-4*a*c)
    return ( ((-b+delta)*(-b-delta)) / ((2*a)*(-b-delta)) , ((-b-delta)*(-b+delta)) / ((2*a)*(-b+delta)) )

def quad_eq_v3(x,y,z):
    a = D(x)
    b = D(y)
    c = D(z)
    delta = D.sqrt((b**2-4*a*c))
    x1 = D((-b+delta)/(2*a))
    x2 = D((-b-delta)/(2*a))
    return format(x1,'.3'), format(x2,'.3')

# a 
x1,x2 = quad_eq(0.001,1000,0.001)
print('a)',x1,x2)

# b 
x1,x2 = quad_eq_v2(0.001,1000,0.001)
print('b)',x1,x2)

# c 
x1,x2 = quad_eq_v3(0.001,1000,0.001)
print('c)',x1,x2)

a) -9.999894245993346e-07 -999999.999999
b) -9.999894245993346e-07 -999999.9999990001
c) -0.00000100 -1.00E+6


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [11]:
def f(x):
    return x*(x-1.)

for delta in [1.*10**(-x) for x in [2*y for y in range(2,8)]]:
    der = (f(1.+delta)-f(1.))/delta # should be 2x-1 => f'(x=1)=1
    print("Delta: ",delta, ", f'(x=1) = ",der)

Delta:  0.0001 , f'(x=1) =  1.0000999999998899
Delta:  1e-06 , f'(x=1) =  1.0000009999177333
Delta:  1e-08 , f'(x=1) =  1.0000000039225287
Delta:  1e-10 , f'(x=1) =  1.000000082840371
Delta:  1e-12 , f'(x=1) =  1.0000889005833413
Delta:  1e-14 , f'(x=1) =  0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [1]:
import math
import numpy as np
import time 

def semicirc(x):
    return math.sqrt(1.-x**2)

def Riemann(N=100, output=False):
    start = time.clock()
    h = 2/N
    I = 0.
    for i in np.linspace(-1,1,N):
            I+=h*semicirc(i)
    if output == True:
        print("Riemann: ",I,'with N = ',N)
        print("Difference with true value", np.fabs(I - math.pi/2))
        print(time.clock() - start, "seconds")
    return time.clock() - start

N = 200000
while Riemann(N) < 1:
    N+=100

Riemann(N,True)

Riemann:  1.5707887863930305 with N =  208800
Difference with true value 7.5404018660130134e-06
0.9420897392081997 seconds


0.9422138993671254

In [5]:
Riemann(N*120,True)

Riemann:  1.5707962640901023 with N =  25056000
Difference with true value 6.270479424586028e-08
65.27988089158086 seconds


65.28008227850532